# 프로필 검색 인덱스 만들기

- 프로필 데이터 로드하기
- index 만들고 저장하고 로드하기
- 검색하기


In [ ]:
!pip install --upgrade --quiet langchain langchain-openai faiss-cpu tiktoken

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

import pandas as pd

# 프로필 데이터 로드하기

In [2]:
df = pd.read_json("./profile_gen_train.jsonl", lines=True, orient="records")

In [3]:
profile_text_list = []

for i, row in df.iterrows():
    profile_text = f"{row['messages'][1]['content']}\n자기소개:{row['messages'][2]['content']}"
    profile_text_list.append(profile_text)

In [4]:
profile_text_list

['이름: 이소라\n나이: 32\n성별: 여자\n직업: 작가\n키워드: 글쓰기, 소통, 영감, 카페\n자기소개:말보다는 글로 소통하는 것을 좋아합니다. 일상에서 영감을 받아 이야기를 써내려가요. 카페에서의 느긋한 오후가 저의 충전소입니다.',
 '이름: 조현우\n나이: 27\n성별: 남자\n직업: 건축가\n키워드: 건축, 디자인, 여행, 예술\n자기소개:공간을 통해 사람들의 삶을 변화시키는 것에 열정을 가지고 있습니다. 여행을 통해 다양한 건축물에서 영감을 받아요.',
 '이름: 김현중\n나이: 35\n성별: 남자\n직업: 마케터\n키워드: 마케팅, 브랜딩, 와인, 음악\n자기소개:시장의 흐름을 읽고, 브랜드의 이야기를 전달하는 것에 재미를 느낍니다. 와인과 음악을 사랑하며, 늘 새로운 취미를 찾아 탐험하는 것을 좋아해요.',
 '이름: 이준희\n나이: 35\n성별: 남자\n직업: 요리사\n키워드: 요리, 와인, 음악, 여행\n자기소개:재료의 본연의 맛을 살려 최고의 요리를 만드는 것에 자부심을 가집니다. 와인과 음악은 저의 다른 열정입니다.',
 '이름: 오민아\n나이: 33\n성별: 여자\n직업: 연구원\n키워드: 연구, 호기심, 등산, 독서\n자기소개:세상을 바꾸는 새로운 발견에 기여하고 싶은 연구원입니다. 과학적 호기심과 탐구 정신이 저를 이끌어요. 등산과 독서로 여가시간을 보냅니다.',
 '이름: 황태현\n나이: 27\n성별: 남자\n직업: 엔지니어\n키워드: 기술, 학습, 캠핑, 소통\n자기소개:기계와 대화하듯, 기술로 세상과 소통하는 엔지니어. 새로운 기술을 배우고 응용하는 것에 열정적입니다. 자연 속에서의 캠핑으로 일상의 스트레스를 해소해요.',
 '이름: 유진아\n나이: 30\n성별: 여자\n직업: 의사\n키워드: 건강, 치유, 요가, 명상\n자기소개:사람의 몸과 마음을 치유하는 것에 가치를 두는 의사입니다. 건강한 생활, 요가와 명상을 일상에 녹여내며, 사람들과의 깊은 대화를 즐깁니다.',
 '이름: 박지혜\n나이: 26\n성별: 여자\n직업

## 검색 인덱스 만들고 저장하기

In [5]:
vectorstore = FAISS.from_texts(
    profile_text_list, embedding=OpenAIEmbeddings()
)


In [6]:
vectorstore.save_local("./profile_index")

## 검색 인덱스 불러오고 검색하기

In [8]:
vectorstore = FAISS.load_local(folder_path="./profile_index", embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [9]:
retriever = vectorstore.as_retriever()

In [10]:
retriever.invoke("글쓰기")

[Document(id='94a5dbbc-2226-4bdd-9b00-9bdacc73140b', metadata={}, page_content='이름: 이소라\n나이: 32\n성별: 여자\n직업: 작가\n키워드: 글쓰기, 소통, 영감, 카페\n자기소개:말보다는 글로 소통하는 것을 좋아합니다. 일상에서 영감을 받아 이야기를 써내려가요. 카페에서의 느긋한 오후가 저의 충전소입니다.'),
 Document(id='8c736007-0a4d-4cd6-9f8d-3d73fee306f2', metadata={}, page_content='이름: 유은지\n나이: 28\n성별: 여자\n직업: 작가\n키워드: 글쓰기, 커피숍, 사람 구경\n자기소개:일상의 소중함을 글로 쓰는 작가입니다. 커피숍에서 사람 구경하는 것을 좋아해요.'),
 Document(id='a303c87d-9033-44ad-a907-5010bff3efb1', metadata={}, page_content='이름: 김태현\n나이: 30\n성별: 남자\n직업: 소프트웨어 엔지니어\n키워드: 코딩, 커피, 여행, 사진\n자기소개:코드와 커피를 사랑하는 개발자입니다. 일상에서 영감을 받아 새로운 프로젝트를 만드는 것이 취미입니다. 여행과 사진 찍기를 좋아해요.'),
 Document(id='5119f4c5-22f9-48f2-a92e-db5ce0133dcc', metadata={}, page_content='이름: 오민아\n나이: 33\n성별: 여자\n직업: 연구원\n키워드: 연구, 호기심, 등산, 독서\n자기소개:세상을 바꾸는 새로운 발견에 기여하고 싶은 연구원입니다. 과학적 호기심과 탐구 정신이 저를 이끌어요. 등산과 독서로 여가시간을 보냅니다.')]

In [11]:
retriever.invoke("카페")

[Document(id='94a5dbbc-2226-4bdd-9b00-9bdacc73140b', metadata={}, page_content='이름: 이소라\n나이: 32\n성별: 여자\n직업: 작가\n키워드: 글쓰기, 소통, 영감, 카페\n자기소개:말보다는 글로 소통하는 것을 좋아합니다. 일상에서 영감을 받아 이야기를 써내려가요. 카페에서의 느긋한 오후가 저의 충전소입니다.'),
 Document(id='a303c87d-9033-44ad-a907-5010bff3efb1', metadata={}, page_content='이름: 김태현\n나이: 30\n성별: 남자\n직업: 소프트웨어 엔지니어\n키워드: 코딩, 커피, 여행, 사진\n자기소개:코드와 커피를 사랑하는 개발자입니다. 일상에서 영감을 받아 새로운 프로젝트를 만드는 것이 취미입니다. 여행과 사진 찍기를 좋아해요.'),
 Document(id='8c736007-0a4d-4cd6-9f8d-3d73fee306f2', metadata={}, page_content='이름: 유은지\n나이: 28\n성별: 여자\n직업: 작가\n키워드: 글쓰기, 커피숍, 사람 구경\n자기소개:일상의 소중함을 글로 쓰는 작가입니다. 커피숍에서 사람 구경하는 것을 좋아해요.'),
 Document(id='d04a93d4-0b11-4664-914e-0c2a7efadca8', metadata={}, page_content='이름: 이소영\n나이: 29\n성별: 여자\n직업: 디자이너\n키워드: 디자인, 캘리그라피, 요리, 베이킹\n자기소개:색감과 디테일에 민감한 디자이너. 캘리그라피로 감성을 표현하는 것을 좋아합니다. 요리와 베이킹으로 사랑을 전해요.')]

In [12]:
retriever.invoke("여행")

[Document(id='59271746-871c-48b8-847f-201f9c6ce7d4', metadata={}, page_content='이름: 최유리\n나이: 26\n성별: 여자\n직업: 영어 강사\n키워드: 언어, 소통, 여행, 문화\n자기소개:언어를 통해 세상과 소통하는 것을 좋아합니다. 여행을 다니며 새로운 문화를 경험하는 것이 취미입니다.'),
 Document(id='a303c87d-9033-44ad-a907-5010bff3efb1', metadata={}, page_content='이름: 김태현\n나이: 30\n성별: 남자\n직업: 소프트웨어 엔지니어\n키워드: 코딩, 커피, 여행, 사진\n자기소개:코드와 커피를 사랑하는 개발자입니다. 일상에서 영감을 받아 새로운 프로젝트를 만드는 것이 취미입니다. 여행과 사진 찍기를 좋아해요.'),
 Document(id='9b48b8ef-f354-4ed8-9685-59403242eb8c', metadata={}, page_content='이름: 이준희\n나이: 35\n성별: 남자\n직업: 요리사\n키워드: 요리, 와인, 음악, 여행\n자기소개:재료의 본연의 맛을 살려 최고의 요리를 만드는 것에 자부심을 가집니다. 와인과 음악은 저의 다른 열정입니다.'),
 Document(id='9115146f-beff-4bf0-a4f5-d29ca3233428', metadata={}, page_content='이름: 조현우\n나이: 27\n성별: 남자\n직업: 건축가\n키워드: 건축, 디자인, 여행, 예술\n자기소개:공간을 통해 사람들의 삶을 변화시키는 것에 열정을 가지고 있습니다. 여행을 통해 다양한 건축물에서 영감을 받아요.')]